# Entrega 2 - Clasificador (Árboles de Decisión)

### Grupo 07:
     - Renzo Gambone C.I. 5.155.486-4
     - Germán Ouviña C.I. 4.823.566-1
     - Leandro Rodríguez C.I 4.691.736-4


## 1. Introducción
***
### 1.1. Objetivo
***
El objetivo de esta tarea fue implementar un modelo de **árboles de decisión**, basándose su entrenamiento en el algoritmo **ID3** con una serie de modificaciones. Una vez generados distintos clasificadores en base a ciertos parámetros, se evaluó la performance de los mismos utilizando distintos tipos de métrica y comparando dichas evaluaciones para determinar cual modelo se ajustó mejor a cada escenario.

En términos formales, los parámetros del problema se reducen a lo siguiente:
- **Tarea *T*:** Clasificar ejemplos siguiendo cierto formato.
- **Experiencia *E*:** Conjunto de datos con ejemplos ya clasificados siguiendo cierto formato.
- **Performance *P*:** Múltiples tipos de medidas (las cuales se profundizan en la sección 3).

### 1.2. Entrega
***
La entrega de esta tarea consta de dos grandes componentes:
- **Informe** en formato de Jupyter Notebook (este informe).
- **Programa** que permite entrenar clasificadores, evaluar su desempeño y clasificar nuevos ejemplos.

El objetivo del informe es centralizar la información relativa a la construcción del modelo así como los datos obtenidos mediante pruebas realizadas con distintas configuraciones paramétricas. Como agregado opcional, se adjuntan algunos scripts para ayudar a la lectura o probar en tiempo real ciertas funcionalidades.

Por otra parte, el programa ofrece una interfaz en consola que permite entrenar y evaluar árboles acorde a múltiples configuraciones paramétricas, mostrar los arboles entrenados y calificar elementos del conjunto asociado al árbol. También se agregó un script para entrenar y evaluar modelos automáticamente. Si bien todas estas herramientas fueron pensadas para uso del grupo, en el archivo README.md se adjunta una sencilla guía de como utilizarlas.

### 1.3. Formato
***
En las siguientes secciones se especifica el diseño del modelo, justificando la toma de decisiones a la hora de construirlo y se detallan las estrategias o algoritmos que fueron implementados para la configuración paramétrica, junto a las métricas utilizadas en la evaluación. Luego, se detalla la metodología de experimentación y con la misma los resultados obtenidos para cada modelo y cada conjunto de datos, habiendo entrenado con distintas configuraciones paramétricas. Finalmente se agregan conclusiones respecto a los resultados obtenidos. 

## 2. Diseño
***
En esta sección se detallan las características del diseño utilizado para construir el modelo, se profundizan las estrategias y algoritmos empleados en la configuración paramétrica y se tratan otros puntos como el procesamiento previo al entrenamiento y la evaluación posterior al mismo.

### 2.1. Modelo
***
La consigna propone la utilización y comparación de dos tipos de modelos que basan su entrenamiento en **ID3**. Dichos modelos serán denominados **árboles de decisión** y **bosques de decisión** respectivamente. A continuación, se especifican las características de cada uno:

**Notas:**
* No se ahonda en dichas descripciones puesto que son conceptos trabajados en el teórico.
* Se utiliza notación matemática para visualizar fácilmente la definición de cada modelo.

#### 2.1.1. Árbol de decisión
***
Para este escenario uno de los modelos propuestos es el de un **árbol de decisión** multiclase, es decir, una estructura de árbol que para un ejemplo dado, lo clasifica en una de múltiples clases.

Se definen más formalmente las siguientes nociones:
* $D =$ Conjunto de entrenamiento
* $C_D =$ Conjunto de posibles clasificaciones para $d \in D$
* $T_D =$ Árbol de decisión generado en base a $D$
* Se representa una clasificación de un ejemplo $d \in D$ realizada por el árbol $T_D$ como $T_D(d) = c : c \in C_D$ y $d \in D$

#### 2.1.2. Bosque de decisión
***

El otro modelo propuesto es el de un **bosque de decisión** multiclase de árboles binarios. El modelo de bosque se basa fuertemente en el modelo de árbol, consistiendo básicamente en un conjunto de árboles de decisión que clasifican un ejemplo para cada clase. El bosque pondera las clasificaciones de cada árbol en base a cierto criterio y genera una clasificación.

Se definen más formalmente las siguientes nociones:
* $D =$ Conjunto de entrenamiento
* $C_D =$ Conjunto de posibles clasificaciones para $d \in D$
* $B_D =$ Bosque de decisión generado en base a $D$
* $B_D(d) = c : c \in C_D ^ d \in D$, Representando esto una clasificación de un ejemplo $d \in D$ realizada por el bosque $B_D$
* $T_{(B, c)} =$ Árbol de decisión binario perteneciente a $B_D$ y generado en base a la clasificación $c \in C_D$
* Sea $|C_D| = n$, hay $n$ árboles de decisión binarios en $B_D$
* Se representa una clasificación de un ejemplo $d \in D$ realizada por el árbol $T_{(B, c)}$ como $T_{(B, c)}(d) = c' : c' \in \{True, False\}$ y $d \in D$

Los detalles sobre los algoritmos de entrenamiento y clasificación utilizados por cada modelo se expanden en la sección 2.3.



### 2.2. Preprocesamiento
***
Con los objetivos tanto de hacer viable, como de mejorar la performance de entrenamiento, se realizaron ciertos cambios en el conjunto de datos a utilizar. A continuación, se listan los cambios realizados agrupandolos en base a motivación y área cambiada en el conjunto original.

**Notas:**
* Los cambios realizados en el conjunto de datos fueron hechos en memoria, sin modificar de forma alguna los archivos originales.

#### 2.2.1. Optimización (Representación)
***
La elección de la representación interna fue compleja, ya que se tuvo en cuenta tanto la performance como la facilidad a la hora de programar e interpretar el código.

Luego de numerosas pruebas utilizando distintas representaciones, se determinó la utilización del objeto **DataFrame** de la biblioteca *pandas*. Al principio trajo muchos problemas de performance (tanto en uso de memoria como en velocidad) pero al implementar técnicas ofrecidas por la biblioteca dichos problemas se palearon, generando eventualmente un uso de memoria reducido y un entrenamiento veloz.

#### 2.2.2. Estructura (Atributos)
***
Originalmente se buscó mantener la estructura de atributos estática, con el fin de impactar lo menos posible en el entrenamiento y su posterior evaluación. No obstante, para el segundo conjunto de datos (detallado en la sección 2.4), fue necesario cambiar la estructura eliminando varios atributos y generando otros nuevos.

Más concretamente, el cambio realizado fue el proceso inverso al conocido como **one hot encoding**, ya que habia múltiples atributos que representaban "la idea" de un sólo atributo discreto. En múltiples problemas, la implementación de dicho algoritmo resulta de gran utilidad para mejorar el rendimiento tanto a nivel de performance como de clasificación, generando $x$ atributos binarios para $x$ posibles valores del atributo original. 

En el contexto de árboles de decisión, sucede lo opuesto: el aumento lineal en cantidad de atributos genera un aumento exponencial en el tamaño del árbol y en el tiempo de entrenamiento. Tomando esto en cuenta, se deshizo el **one hot encoding**, tratando a los atributos generados como continuos (este punto junto a sus implicancias son expandidos en las siguientes secciones).

#### 2.2.3. Clasificación (Resultados)
***
Para el modelo de **árbol de decisión** no fue necesario realizar ningún cambio en el conjunto de datos. No obstante, para entrenar a los árboles pertenecientes al **bosque de decisión**, al ser estos de carácter binario y funcionar para una sola clase, hubo que modificar los resultados del conjunto.

Utilizando las nociones de la sección 2.1, se procesó el conjunto de datos generando $|C_D| = n$ conjuntos con las clasificaciones correspondientes, uno para cada posible valor. Cada árbol del bosque fue entrenado utilizando uno de esos conjuntos generados. 

El conjunto original no fue modificado, por lo que el bosque en sí puede clasificar para las clases originales.

### 2.3. Algoritmo
***
En la siguiente sección se centralizan todas las especificaciones relativas a los algoritmos empleados, tanto para entrenar como para clasificar. 

**Notas:**
* Se hace referencia a puntos mencionados en secciones anteriores, sin entrar en detalle en lo que ya fue explicado.

#### 2.3.1. Algoritmo de entrenamiento
***
El algoritmo de entrenamiento se basa en el conocido **algoritmo ID3** para construcción de **árboles de decisión**. Dicho algoritmo sigue las pautas generales vistas en el teórico. Se agregó la posibilidad de tratar atributos continuos y se varió el uso de medidas para detectar el mejor atributo. Dichos cambios se tratan en las sub secciones 2.3.3 y 2.3.4. 

Teniendo en cuenta lo mencionado en secciones anteriores, el algoritmo de entrenamiento implementado puede entrenar dos tipos de modelos, generando dos tipos de clasificadores:

- **Árbol de decisión:** Se implementa el entrenamiento greedy estándar de ID3 con los cambios anteriormente mencionados y se obtiene como resultado un árbol de decisión multiclase.
- **Bosque de decisión:** Se itera sobre cada posible clasificación, procesando el conjunto de datos según corresponda y generando un árbol de decisión binario utilizando el algoritmo ID3 adaptado anteriormente mencionado.

#### 2.3.2. Algoritmo de clasificación
***
A la hora de clasificar un elemento como miembro de una clase, el procedimiento difiere dependiendo del modelo utilizado. No obstante, al igual que con el entrenamiento, la clasificación de un bosque se basa fuertemente en la de un árbol. 

Es importante mencionar que los árboles generados tienen una **probabilidad** asignada a cada hoja. Dicha probabilidad es calculada al momento de generar la hoja durante el entrenamiento. Al no quedar más atributos, se calcula la frecuencia de cada clase en el subconjunto de datos que hay en la hoja y aquella clasificación con mayor frecuencia es la elegida por la hoja, adjuntandose dicha frecuencia junto a ella.

Teniendo esto en cuenta, a grandes rasgos, los algoritmos funcionan como sigue:

- **Árbol de decisión:** Se toma el ejemplo a clasificar y se recorre recursivamente los nodos del árbol hasta llegar a una hoja. Se devuelve la clasificación junto a su probabilidad. Por la naturaleza del algoritmo, dicha probabilidad suele ser mayor a 0.5. 
- **Bosque de decisión:** Se toma el ejemplo a clasificar y se itera sobre cada árbol, obteniendo la clasificación del mismo. Una vez obtenidas todas las clasificaciones, se procede a realizar una **votación**. El resultado de dicha votación es la clasificación del ejemplo en cuestión.

La clasificación del **árbol de decisión** no cuenta con grandes especificidades, ya que es la misma utilizada por los árboles de decisión genéricos. No obstante, para determinar el algoritmo de votación del **bosque de decisión** fue necesario determinar un criterio, por lo que a continuación se explaya sobre este asunto.

En el escenario donde todos los árboles indican *False*, aludiendo a que el elemento no es de su clasificación y uno de los árboles indica *True*, la clasificación es trivial. En este caso el resultado final es la clasificación del elemento. Sin embargo este no es siempre el caso, dado que los árboles de decisión si bien son resistentes al ruido, no son inmunes al mismo.

Un problema a resolver a la hora de trabajar con bosques son los *empates*, en donde uno o más árboles devuelven *True* para determinado ejemplo. El desempate se alcanza comparando la **probabilidad** con la que cada árbol clasificó al ejemplo y tomando aquella clasificación con mayor frecuencia. 

En caso de que haya empate entre varias clasificaciones y sus probabilidades, el bosque elige una de ellas aleatoriamente. En la sección 4 se expanden las implicancias de este hecho.

#### 2.3.3. Selección de mejor atributo
***
Un paso crucial del algoritmo es decidir que atributo "bifurca mejor" el árbol. ID3 propone elegir el atributo que maximice la **ganancia de información**. Sin embargo, esta métrica favorece los atributos que toman múltiples valores sobre otros. A raiz de esto es que también se experimentó con otras estrategias en la configuración paramétrica, como el **ratio de ganancia** y la **reducción de "impureza"**

Todas las siguientes estrategias trabajan con una métrica que determina *"cuánto un atributo separa a los ejemplos según la clasificación objetivo"*, esta métrica es la **entropía** y se define como:
- Sea $D = $ Conjunto de datos
- Sea $C_D = $ Conjunto de posibles clasificaciones para $d \in D$
- Sea $|C_D| = n =$ Cantidad de posibles clasificaciones en $C_D$
- Sea $f_D : D \rightarrow C_D =$ Función que clasifica elementos en $D$
- Sea $p_i$ la proporción de ejemplos $d \in D : f_D(d) = c_i$ con $c_i \in C_D$ la i-ésima clasificación

$$Entropía(D) = -\sum p_i log_2(p_i)$$

La entropía mide la heterogeneidad de los datos: cuanto más homogéneos, menor será la misma.

A continuación, se adjunta una breve noción de las tres medidas utilizadas para determinar *"el mejor atributo"*:

##### 2.3.3.1. Ganancia
***
Se define la **ganancia de información** de un atributo $a$ sobre una muestra $D$ como:

$$Ganancia(D,a) = Entropía(D) - \sum_{v \in Val(a)}\frac{|D_v|}{|D|}\cdotp
 Entropía(D_v)\$$

La fundamentación de tomar la ganancia es que se desea que el atributo sobre el cual se particione divida el conjunto de datos de la forma más homogénea en cuanto a su clasificación posible.

Como se mencionó anteriormente, una desventaja de la ganancia es que favorece a los atributos que asumen un espectro muy alto de valores distintos con respecto a otros atributos. Un conjunto amplio y uniformemente distribuido de valores para el atributo no necesariamente es una característica que debería formar parte a la hora de determinar el mejor atributo para bifurcar.

##### 2.3.3.2. Ratio de ganancia
***
Se define el **ratio de ganancia** de un atributo $a$ sobre una muestra $D$, en función de la **separación de información** de un atributo $a$ sobre una muestra $D$. Dichas definiciones son las siguientes:

$$SeparacionDeInformacion(D,a) = - \sum_{v \in Val(a)}\frac{|D_v|}{|D|} \cdotp log_2(\frac{|D_v|}{|D|})$$

$$RatioGanancia(D,Aa) = \frac{Ganancia(D,a)}{SeparacionDeInformacion(D,a)}$$

La **separación de información** es sensible a que tan amplia y uniformemente el atributo distribuye sus datos, logrando así penalizar a los atributos que tomen un espectro muy amplio de valores. Una colección de ejemplos $D : |D| = n$ que estén completamente separados por un atributo $a_1$ (como una fecha) tendrá $SeparacionDeInformacion(D,a_1) = log_2(n)$. En cambio un atributo $a_2$ (booleano, por ejemplo) que divida los mismos $n$ ejemplos a la mitad tendrá $SeparacionDeInformacion(D,a_2) = 1$. Si ambos atributos producen la misma ganancia de información, entonces $a_2$ tendrá un **ratio de ganancia** más alto.

Se observa que la **separación de información** es la entropía de $D$ respecto a los valores del atributo $a$ (a diferencia de tomarla con respecto a la clasificación).

Una desventaja de usar el **ratio de ganancia** en lugar de sólo **ganancia** es que la **separación de información** puede ser 0, lo que hace el **ratio de ganancia** sea indefinido o muy grande para atributos que pueden tener el mismo valor para casi todos los miembros de $D$.

##### 2.3.3.3. Reducción de "impureza"
***
Se define la **reducción de impureza** de un atributo $a$ sobre una muestra $D$, utilizando la **impureza Gini** (nombrada en honor a Conrado Gini) de una muestra $D$. Dichas definiciones son las siguientes:

$$Gini(D) = 1 - \sum p_{i}^2$$

$$ReductionImpureza(D,a) = Gini(D) - \sum_{v \in Val(a)}\frac{|D_v|}{|D|} \cdotp Gini(D_v)$$

La **impureza Gini**, nombrada por el matemático Corrado Gini, mide la homogeneidad de un conjunto de elementos, tomando un valor mínimo de 0.0 y un valor máximo acotado por 1.0.

Dicha métrica toma valor 0 cuando todos los elementos del conjunto califican al mismo valor. En este contexto se dice que la **impureza** es mínima. A su vez si todos los valores del conjunto califican la misma cantidad de resultados distintos, se dice que la **impureza** es máxima (tomando un valor menor o igual a 1.0).

A los efectos practicos de la construcción de **árboles de decisión**, durante la misma se desea bifurcar con atributos que reduzcan la **impureza** lo mayor posible posible (optando por atributos que califiquen la mayor cantidad de instancias en la misma clase).

Una muy importante observación es que si bien la **impureza Gini** minimiza errores de clasificación y la **entropía** se usa para análisis exploratorio, ambas cumplen un rol casi idéntico. Usualmente, la **impureza Gini** da mejores resultados para atributos continuos mientras que la **entropía** funciona mejor con atributos discretos, sin embargo algunos estudios encontraron que sólo un 2% de las veces los resultados son distintos.

Debido a que su efectividad es dependiente del problema, se decidió incluir reducción de impureza como parte de la configuración paramétrica. 

#### 2.3.4. Interpretación de valores continuos
***
Una característica clave para el escenario propuesto y que **ID3** no contempla es **el tratamiento de atributos continuos**. Se emplearon tres estrategias distintas para interpretar los valores de tales atributos, teniendo como fin las mismas, discretizar el espectro continuo en una pequeña cantidad de clases.

Como noción general independiente a la estrategia implementada, cada clase discreta generada a partir de la estrategia en cuestión representa un **intervalo**. Los intervalos se definen por su **tope superior (máximo)**, excepto el último, que se define por su **tope inferior (mínimo)**

Sea por ejemplo, la lista de intervalos $[v_0, ..., v_n, v_{n+1}]$, para cada valor $v_i : i \in [0..n+1]$ se cumple lo siguiente:
- El intervalo de $v_0$ representa "todos los valores $k : k \leq v_0$".
- El intervalo de $v_i : i \in [1..n]$ representa "todos los valores $k : v_{i-1} \lt k \leq v_i$".
- El intervalo de $v_{n+1}$ representa "todos los valores $k : k \gt v_{n+1}$".

Es importante mencionar que los últimos dos **valores** cumplen ser iguales ($v_n = v_{n+1}$), lo que cambia es lo que el intervalo representa.

Habiendo visto esto, se resumen las estrategias implementadas:

##### 2.3.4.1. Partir en intervalos fijos
***
Como primera estrategia, se determina la siguiente:
- Sea $A_D =$ Conjunto de atributos en conjunto de datos $D$
- Sea $a \in A_D : Val(a) \subseteq N, Z, R$ un atributo continuo
- Sea $a^{*} \in A_D : |Val(a^{*})| \lt \infty$ la discretización de un atributo continuo
- Sea $\mu_{a}$ la mediana de $Val(a)$ para los ejemplos en $D$
- Entonces $Val(a^{*}) = \{\leq \mu_{a}, \gt \mu_{a}\}$

En lenguaje natural, se generan dos intervalos: los valores menores o iguales a la mediana de los valores actuales, y los valores mayores a dicha mediana. Todo nuevo valor entra siempre en alguno de los dos intervalos.

La fundamentación de tomar dos intervalos partidos por la mediana es que es una manera intuitiva de distribuir el atributo de forma "pareja" en el árbol. Por otra parte, la desventaja de esta estrategia está en que, además de no ser necesariamente el punto ideal de corte para la clasificación, el partir en solamente dos intervalos para atributos con múltiples valores puede terminar agrupando ejemplos e introduciendo ruido al conjunto de datos.

##### 2.3.4.2. Partir en intervalos variables
***
Con el fin de mejorar el defecto del enfoque anterior se implementa la siguiente estrategia:
- Sea $D_a =$ Conjunto de datos ordenado ascendentemente en base a atributo $a$
- Sea $A_{D_a} =$ Conjunto de atributos en conjunto de datos $D_a$
- Sea $a \in A_{D_a} : Val(a) \subseteq N, Z, R$ un atributo continuo
- Sea $a^{*} \in A_{D_a} : |Val(a^{*})| \lt \infty$ la discretización de un atributo continuo
- Sea $v_i : v_i \in Val(a)$ el valor de $a$ en el k-ésimo ejemplo $d_k \in D_a$ tal que cambia su clasificación respecto a $d_{k-1}$
- Entonces $Val(a^{*}) = \{\leq v_0, ..., \leq v_n, \gt v_n\}$

En lenguaje natural, se genera un número variable de intervalos. El procedimiento para generarlos es:
1. Ordenar $D$ en base a $a$ (generando $D_a$).
2. Iterar sobre $D_a$
3. Almacenar en $Val(a^{*})$, el valor $v_i$ de $a$ en $d_k$, en el momento en que la clasificación cambia del ejemplo $d_{k-1}$ al ejemplo $d_k$.

De esta forma, se tiene una cantidad de intervalos que relaciona el cambio en la clasificación del ejemplo con el cambio en el atributo $a$. Esta técnica impacta menos negativamente que la anterior en relación al ruido que introduce al recortar valores.

No obstante, en conjuntos de datos muy grandes con valores de $a$ muy dispersos o con clasificaciones muy cambiantes, puede suceder que se genere una cantidad de intervalos cercana a la cantidad de valores originales. Este hecho atenta en contra de la idea de interpretar valores continuos y de construir árboles simples con pocas ramas. No solamente eso, si no que también impacta en el tiempo de entrenamiento.

Teniendo esto en cuenta, se limitó la cantidad de posibles intervalos a 10, eligiendo 10 valores que se separan uniformemente en el conjunto total de intervalos.

##### 2.3.4.3. Partir por el valor que maximice la ganancia
***
Basado en el algortimo C4.5, el algortimo sucesor de ID3, se implementa la siguiente estrategia:
- Sea $D_a =$ Conjunto de datos ordenado ascendentemente en base a atributo $a$
- Sea $A_{D_a} =$ Conjunto de atributos en conjunto de datos $D_a$
- Sea $a \in A_{D_a} : Val(a) \subseteq N, Z, R$ un atributo continuo
- Sea $a^{*} \in A_{D_a} : |Val(a^{*})| \lt \infty$ la discretización de un atributo continuo
- Sea $v_i : v_i \in Val(a)$ el valor de $a$ en el k-ésimo ejemplo $d_k \in D_a$ tal que cambia su clasificación respecto a $d_{k-1}$
- Sea $Val(a') = \{\leq v_0, ..., \leq v_n, \gt v_n\}$ los posibles valores determinados en la sección anterior
- Entonces $Val(a^{*}) = \{\leq v_j, \gt v_j\} : v_j \in Val(a')$ y se maximiza la ganancia de partir el conjunto de datos en $v_j$

En lenguaje natural, se generan dos intervalos como en la sección 2.3.4.1 pero se parte de los valores tomados en la sección 2.3.4.2 y se determina cual de estos otorga una mayor ganancia.

Una desventaja de esto es que es de un mayor orden computacional, requiriendo una recorrida del dataset para calcular la ganancia de cada punto de corte a probar.


### 2.4. Evaluación
***
En la siguiente sección se centralizan todas las especificaciones relativas a la evaluación de los clasificadores entrenados.


#### 2.4.1. Conjuntos de evaluación
***
Los dos conjuntos de datos a partir de los cuales se entrenó y evaluó los modelos son:
- [**Conjunto *Iris*:**](https://archive.ics.uci.edu/ml/datasets/iris) El cual clasifica plantas del genero Iris (de la tribu Irideae, perteneciente a la familia Iridaceae) según especie. Cuenta con 150 ejemplos, 4 atributos de carácter continuo y 3 posibles clasificaciones.
- [**Conjunto *Covertype*:**](https://archive.ics.uci.edu/ml/datasets/Covertype) El cual clasifica terrenos de bosques según atributos cartográficos. Cuenta con 581012 ejemplos,54 atributos, siendo 10 de carácter continuo y 44 de carácter binario (son el one hot encoding de 2 atributos generando 4 y 40 atributos respectivamente) y 7 posibles clasificaciones.

Cabe destacar que el conjunto *Iris* cuenta con una cantidad ínfima de ejemplos, mientras que el conjunto *CoverType* va hacia el otro extremo. Las implicancias de estos hechos se tratan en la sección 3.

#### 2.4.2. Métodos de evaluación
***
Se experimentó utilizando dos estrategias de evaluación, siendo las mismas:
- Validación 80/20 (denominada "normal" a partir de ahora)
- Validación cruzada con 10 particiones (denominada "cruzada" a partir de ahora)

Ambas estrategias subdividen el conjunto de datos $D$ en dos subconjuntos, $T, E \subset D$. El subconjunto $T$ se utiliza para entrenar al clasificador en cuestión, mientras que el subconjunto $E$ se utiliza para evaluar la eficacia del mismo. Esto se logra haciendo que el modelo entrenado clasifique los ejemplos del conjunto de evaluación $E$, comparando luego la clasificación original con la realizada. Dicha comparación se basa en distintas métricas, las cuales se tratan en la sub sección siguiente.

Es importante destacar que, independiente al método de evaluación, el conjunto de datos $D$ original se mezcla de forma aleatoria antes de subdividirlo en $T$ y $E$. Esto es vital para evitar el mismo resultado en distintas evaluaciones.

Si, por ejemplo, $D$ estuviera ordenado de forma tal que los últimos ejemplos en él (los que corresponderían a $E$) no agregan información nueva a $T$, entonces el desempeño del clasificador sería excelente. Por otra parte, si sucede lo contrario, el desempeño sería mucho peor. La mezcla al azar de $D$ se utiliza con el objetivo de reducir este fenómeno.

A continuación se adjunta una breve descripción de los parámetros utilizados en cada método de evaluación.

##### 2.4.2.1. Validación normal
***
Se denomina como **validación normal** a la partición única del conjunto $D$ en $T$ y $E$ utilizando una proporción donde $|T| \lt |E|$.

En general, dependiendo de la cantidad de ejemplos en $D$ y del objetivo que se quiera alcanzar, dicha partición sigue una relación 80/20, 85/15, 90/10, etc. Para el escenario actual, se determinó utilizar una partición 80/20, es decir, $T$ cuenta con el 80% de ejemplos en $D$, mientras que $E$ cuenta con el 20% restante.

Es importante recordar el fenómeno mencionado en la subsección anterior. La validación normal, si bien parte al conjunto luego de haber sido reordenado aleatoriamente, puede verse afectada, por casualidad, por el fenómeno en cuestión. Con el objetivo de reducir esta posibilidad, se agrega el siguiente método

##### 2.4.2.2. Validación cruzada
***

Se denomina como **validación cruzada** a la partición del conjunto $D$ en $k$ particiones $T_i$ y $E_i : i \in [1..k]$,  utilizando una proporción $\frac{k-1}{k}$ y $\frac{1}{k}$ para $T_i$ y $E_i$ respectivamente.

En lenguaje natural, la validación cruzada utiliza un parámetro $k$, generando $k$ particiones de $D$ siguiendo el método normal mencionado en la anterior sub sección. Esto se hace con el objetivo de evitar el sobreajuste en subconjuntos $T \subset D$ específicos. Una vez se hacen las $k$ evaluaciones, se promedian las métricas obtenidas para obtener una mejor noción del desempeño del modelo.

Esta estrategia es más robusta que la **validación normal** anteriormente mencionada, sin embargo tiene el problema de tener un mayor orden computacional, dado que tiene que correr $k$ entrenamientos en lugar de uno.

En la experimentación se decidió fijar $k=10$.

#### 2.4.3. Métricas de evaluación
***
Se utilizaron múltiples métricas para determinar la "calidad" o "eficacia" de los clasificadores entrenados. A continuación, una breve descripción de las mismas:

##### 2.4.3.1 Accuracy
***
La **accuracy** o **exactitud** hace referencia a la cantidad de aciertos que el clasificador tuvo 

##### 2.4.3.2 Matriz de Confusión
***
La **matriz de confusión** es una estructura que permite almacenar todas las clasificaciones realizadas en una evaluación en base a su resultado y su valor original. Su estructura es la siguiente:

- Sea $D =$ Conjunto de datos
- Sea $C_D =$ Conjunto de clasificaciones para $d \in D$ 
- Sea $|C_D| = n$ cantidad de posibles clasificaciones en $C_D$
- La matriz de confusión para una evaluación se define como:
$$M = 1$$

##### 2.4.3.3 Precision, Recall, Fall-off, F-measure
***
A

##### 2.4.3.4 Medidas macro y micro
***
A




## 3. Experimentación
***
En esta sección se detalla el conjunto de pruebas realizadas, agregandose observaciones pertinentes y comparaciones entre modelos.

### 3.1. Metodología
***

Con el objetivo de evaluar las estrategias planteadas anteriormente y su efectividad, se probaron combinaciones de las mismas frente a los dos modelos desarrollados. La exprimentación se encaró diviendola en las siguientes etapas:

1. **Recopilación y ajuste:** Si bien las configuraciones paramétricas y otras opciones generales eran conocidas desde una etapa bastante temprana, hubo ciertos cambios realizados a ensayo y error antes de llegar a un algoritmo óptimo. En esta sub sección se repasa brevemente el proceso de construcción previo a la evaluación de modelos terminados.
<br><br>
2. **Configuraciones paramétricas:** Luego de llegar a un modelo relativamente libre de errores (al menos de errores identificados), se realizaron pruebas para múltiples configuraciones paramétricas y comparar cuales de ellas resultaron ser las mejores. En esta etapa se detallan los datos obtenidos. 
<br><br>
3. **Elección y comparación de representantes:** A falta de suficientes corridas para generar agrupamientos de configuraciones, se compararon manualmente los resultados obtenidos en las pruebas de cada configuración paramétrica de la etapa anterior. En base a distintas métricas, se eligió a aquellos clasificadores con mejores y peores resultados, interpretando la razón de su eficacia.

### 3.2. Recopilación y ajuste
***

Aca hablo yo de como llegamos al algoritmo base que funca

- Primera version: Covertype no termina
- Segunda version: Optimización de loops, poda. Iris parece sobreajustar menos pero Covertype no termina. 
- Tercera version: Optimización de pandas y loops, eliminación de poda. Iris funciona como antes y Covertype termina

### 3.3. Configuraciones paramétricas
***
A continuación se 

Es importante destacar que a causa de el tamaño del conjunto *Covertype*, no fue posible probar todas las configuraciones paramétricas para el mismo. De hecho, todas las configuraciones que tratan atributos continuos como **intervalos variables** fueron omitidas, ya que el árbol crecía exponencialmente, creciendo también de esta forma el tiempo de entrenamiento.

#### 3.3.1. Evaluaciones
***
Acá hablo yo

##### 3.3.1.1 Conjuntos de datos
***
Iris y Covertype (2 opciones) Acá hablo yo

##### 3.3.1.2 Modelos
***
Árbol y Bosque (2 opciones)

##### 3.3.1.3 Parámetros
***
Fijo, Variable, C45 (3 opciones)
Ganancia, Ratio de Ganancia, Reducción de Impureza (3 opciones)
Acá hablo yo

#### 3.3.2. Resultados
***
Acá hablo yo


In [1]:
import getConfusionMatrix.py

Box(children=(Box(children=(Dropdown(options=('Conjunto: Iris', 'Conjunto: Covertype'), value='Conjunto: Iris'…



##### 3.3.3.1. Matrices de Confusión
***
Introducción



##### 3.3.3.2. Comparación de resultados
***

Introducción

<table>
    <tr>
        <th>Modelo</th>
        <th>Continuos</th>
        <th>Medida</th>
        <th>|</th>
        <th>Accuracy</th>
        <th>Precision promediada</th>
        <th>Precision ponderada</th>
        <th>Recall promediada</th>
        <th>Recall ponderada</th>
        <th>Fall-off promediada</th>
        <th>Fall-off ponderada</th>
        <th>F-measure promediada</th>
        <th>F-measure ponderada</th>
    </tr>
    <tr>
        <td>Árbol</td>
        <td>Fijo</td>
        <td>Ganancia</td>
        <td>|</td>
        <td>0.873</td>
        <td>0.884</td>
        <td>0.898</td>
        <td>0.889</td>
        <td>0.873</td>
        <td>0.059</td>
        <td>0.060</td>
        <td>0.873</td>
        <td>0.875</td>
    </tr>
    <tr>
        <td>Árbol</td>
        <td>Fijo</td>
        <td>Ratio de Ganancia</td>
        <td>|</td>
        <td>0.893</td>
        <td>0.897</td>
        <td>0.923</td>
        <td>0.906</td>
        <td>0.893</td>
        <td>0.051</td>
        <td>0.048</td>
        <td>0,879</td>
        <td>0,893</td>
    </tr>
    <tr>
        <td>Árbol</td>
        <td>Fijo</td>
        <td>Reducción de Impureza</td>
        <td>|</td>
        <td>0.886</td>
        <td>0.880</td>
        <td>0.920</td>
        <td>0.905</td>
        <td>0.886</td>
        <td>0.050</td>
        <td>0.039</td>
        <td>0,872</td>
        <td>0,888</td>
    </tr>
    <tr>
        <td>Árbol</td>
        <td>Variable</td>
        <td>Ganancia</td>
        <td>|</td>
        <td>0.906</td>
        <td>0.918</td>
        <td>0.926</td>
        <td>0.912</td>
        <td>0.906</td>
        <td>0.046</td>
        <td>0.047</td>
        <td>0,903</td>
        <td>0,906</td>
    </tr>
    <tr>
        <td>Árbol</td>
        <td>Variable</td>
        <td>Ratio de Ganancia</td>
        <td>|</td>
        <td>0.933</td>
        <td>0.939</td>
        <td>0.944</td>
        <td>0.938</td>
        <td>0.933</td>
        <td>0.033</td>
        <td>0.037</td>
        <td>0,932</td>
        <td>0,933</td>
    </tr>
    <tr>
        <td>Árbol</td>
        <td>Variable</td>
        <td>Reducción de Impureza</td>
        <td>|</td>
        <td>0.920</td>
        <td>0.931</td>
        <td>0.947</td>
        <td>0.932</td>
        <td>0.920</td>
        <td>0.037</td>
        <td>0.030</td>
        <td>0,917</td>
        <td>0,919</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>Árbol</td>
        <td>Maximizando ganancia</td>
        <td>Ganancia</td>
        <td>|</td>
        <td>0.953</td>
        <td>0.958</td>
        <td>0.964</td>
        <td>0.950</td>
        <td>0.953</td>
        <td>0.022</td>
        <td>0.019</td>
        <td>0,948</td>
        <td>0,953</td>
    </tr>
    <tr>
        <td>Árbol</td>
        <td>Maximizando ganancia</td>
        <td>Ratio de ganancia</td>
        <td>|</td>
        <td>0.946</td>
        <td>0.953</td>
        <td>0.955</td>
        <td>0.950</td>
        <td>0.946</td>
        <td>0.026</td>
        <td>0.025</td>
        <td>0,946</td>
        <td>0,946</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>Árbol</td>
        <td>Maximizando ganancia</td>
        <td>Reducción de Impureza</td>
        <td>|</td>
        <td>0.953</td>
        <td>0.954</td>
        <td>0.957</td>
        <td>0.952</td>
        <td>0.953</td>
        <td>0.023</td>
        <td>0.025</td>
        <td>0,951</td>
        <td>0,953</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Fijo</td>
        <td>Ganancia</td>
        <td>|</td>
        <td>0.913</td>
        <td>0.930</td>
        <td>0.932</td>
        <td>0.905</td>
        <td>0.913</td>
        <td>0.044</td>
        <td>0.046</td>
        <td>0,906</td>
        <td>0,912</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Fijo</td>
        <td>Ratio de Ganancia</td>
        <td>|</td>
        <td>0.906</td>
        <td>0.909</td>
        <td>0.925</td>
        <td>0.903</td>
        <td>0.906</td>
        <td>0.045</td>
        <td>0.044</td>
        <td>0,892</td>
        <td>0,905</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Fijo</td>
        <td>Reducción de Impureza</td>
        <td>|</td>
        <td>0.906</td>
        <td>0.913</td>
        <td>0.934</td>
        <td>0.918</td>
        <td>0.906</td>
        <td>0.042</td>
        <td>0.034</td>
        <td>0,900</td>
        <td>0,908</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Variable</td>
        <td>Ganancia</td>
        <td>|</td>
        <td>0.913</td>
        <td>0.904</td>
        <td>0.932</td>
        <td>0.915</td>
        <td>0.913</td>
        <td>0.040</td>
        <td>0.035</td>
        <td>0,900</td>
        <td>0,915</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Variable</td>
        <td>Ratio de Ganancia</td>
        <td>|</td>
        <td>0.900</td>
        <td>0.908</td>
        <td>0.914</td>
        <td>0.903</td>
        <td>0.900</td>
        <td>0.047</td>
        <td>0.046</td>
        <td>0,898</td>
        <td>0,899</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Variable</td>
        <td>Reducción de Impureza</td>
        <td>|</td>
        <td>0.880</td>
        <td>0.869</td>
        <td>0.897</td>
        <td>0.882</td>
        <td>0.880</td>
        <td>0.056</td>
        <td>0.056</td>
        <td>0,866</td>
        <td>0,881</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Maximizando ganancia</td>
        <td>Ganancia</td>
        <td>|</td>
        <td>0.946</td>
        <td>0.945</td>
        <td>0.953</td>
        <td>0.950</td>
        <td>0.946</td>
        <td>0.025</td>
        <td>0.024</td>
        <td>0,944</td>
        <td>0,946</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Maximizando ganancia</td>
        <td>Ratio de ganancia</td>
        <td>|</td>
        <td>0.927</td>
        <td>0.919</td>
        <td>0.941</td>
        <td>0.915</td>
        <td>0.926</td>
        <td>0.038</td>
        <td>0.042</td>
        <td>0,906</td>
        <td>0,926</td>
    </tr>
    <tr style="background-color: #afdb99;">
        <td>Bosque</td>
        <td>Maximizando ganancia</td>
        <td>Reducción de Impureza</td>
        <td>|</td>
        <td>0.960</td>
        <td>0.965</td>
        <td>0.964</td>
        <td>0.953</td>
        <td>0.960</td>
        <td>0.021</td>
        <td>0.024</td>
        <td>0,955</td>
        <td>0,959</td>
    </tr>
    <caption>Tabla 1 - Resultados de entrenamiento de conjunto <b>Iris</b> para cada configuración paramétrica</caption>
</table>

<table>
    <tr>
        <th>Modelo</th>
        <th>Continuos</th>
        <th>Medida</th>
        <th>|</th>
        <th>Accuracy</th>
        <th>Precision promediada</th>
        <th>Precision ponderada</th>
        <th>Recall promediada</th>
        <th>Recall ponderada</th>
        <th>Fall-off promediada</th>
        <th>Fall-off ponderada</th>
        <th>F-measure promediada</th>
        <th>F-measure ponderada</th>
    </tr>
    <tr>
        <td>Árbol</td>
        <td>Fijo</td>
        <td>Ganancia</td>
        <td>|</td>
        <td>0.635</td>
        <td>0.658</td>
        <td>0.638</td>
        <td>0.385</td>
        <td>0.635</td>
        <td>0.074</td>
        <td>0.222</td>
        <td>0,434</td>
        <td>0,620</td>
    </tr>
    <tr>
        <td>Árbol</td>
        <td>Fijo</td>
        <td>Ratio de Ganancia</td>
        <td>|</td>
        <td>0.685</td>
        <td>0.682</td>
        <td>0.684</td>
        <td>0.430</td>
        <td>0.685</td>
        <td>0.066</td>
        <td>0.195</td>
        <td>0,481</td>
        <td>0,670</td>
    </tr>
    <tr>
        <td>Árbol</td>
        <td>Fijo</td>
        <td>Reducción de Impureza</td>
        <td>|</td>
        <td>0.633</td>
        <td>0.619</td>
        <td>0.633</td>
        <td>0.390</td>
        <td>0.633</td>
        <td>0.075</td>
        <td>0.225</td>
        <td>0,436</td>
        <td>0,618</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>Árbol</td>
        <td>Maximizando ganancia</td>
        <td>Ganancia</td>
        <td>|</td>
        <td>0.654</td>
        <td>0.651</td>
        <td>0.653</td>
        <td>0.527</td>
        <td>0.654</td>
        <td>0.069</td>
        <td>0.198</td>
        <td>0,555</td>
        <td>0,648</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>Árbol</td>
        <td>Maximizando ganancia</td>
        <td>Ratio de ganancia</td>
        <td>|</td>
        <td>0.693</td>
        <td>0.654</td>
        <td>0.692</td>
        <td>0.491</td>
        <td>0.693</td>
        <td>0.062</td>
        <td>0.171</td>
        <td>0,518</td>
        <td>0,683</td>
    </tr>
    <tr style="background-color: #afdb99;">
        <td>Árbol</td>
        <td>Maximizando ganancia</td>
        <td>Reducción de Impureza</td>
        <td>|</td>
        <td>0.655</td>
        <td>0.652</td>
        <td>0.653</td>
        <td>0.531</td>
        <td>0.655</td>
        <td>0.069</td>
        <td>0.197</td>
        <td>0,556</td>
        <td>0,648</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Fijo</td>
        <td>Ganancia</td>
        <td>|</td>
        <td>0.609</td>
        <td>0.339</td>
        <td>0.651</td>
        <td>0.390</td>
        <td>0.609</td>
        <td>0.071</td>
        <td>0.154</td>
        <td>0,344</td>
        <td>0,621</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Fijo</td>
        <td>Ratio de Ganancia</td>
        <td>|</td>
        <td>0.600</td>
        <td>0.332</td>
        <td>0.640</td>
        <td>0.389</td>
        <td>0.600</td>
        <td>0.073</td>
        <td>0.162</td>
        <td>0,334</td>
        <td>0,610</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Fijo</td>
        <td>Reducción de Impureza</td>
        <td>|</td>
        <td>0.607</td>
        <td>0.347</td>
        <td>0.648</td>
        <td>0.414</td>
        <td>0.607</td>
        <td>0.072</td>
        <td>0.156</td>
        <td>0,353</td>
        <td>0,618</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Maximizando ganancia</td>
        <td>Ganancia</td>
        <td>|</td>
        <td>0.681</td>
        <td>0.416</td>
        <td>0.705</td>
        <td>0.459</td>
        <td>0.681</td>
        <td>0.060</td>
        <td>0.136</td>
        <td>0,429</td>
        <td>0,689</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Maximizando ganancia</td>
        <td>Ratio de ganancia</td>
        <td>|</td>
        <td>0.680</td>
        <td>0.411</td>
        <td>0.692</td>
        <td>0.433</td>
        <td>0.680</td>
        <td>0.062</td>
        <td>0.152</td>
        <td>0,416</td>
        <td>0,681</td>
    </tr>
    <tr>
        <td>Bosque</td>
        <td>Maximizando ganancia</td>
        <td>Reducción de Impureza</td>
        <td>|</td>
        <td>0.689</td>
        <td>0.438</td>
        <td>0.711</td>
        <td>0.483</td>
        <td>0.689</td>
        <td>0.059</td>
        <td>0.137</td>
        <td>0,454</td>
        <td>0,696</td>
    </tr>
    <caption>Tabla 2 - Resultados de entrenamiento de conjunto <b>CoverType</b> para cada configuración paramétrica</caption>
</table>

### 3.4. Elección y comparación de representantes
***

#### 3.4.1. Elección
***
Para elegir los representantes con los cuales comparar se siguió el siguiente proceso:
- 1) Se agruparon elementos en subgrupos dada su similutud en: Accuracy y promedios generales de Precisión, Recall, FallOff y F-measure
- 2) Los subgrupos fueron refinados en más subgrupos acorde a comparaciones entre otros subgrupos y los promedios ponderados.
- 3) Se observaron desviaciones entre los mismos, en base a estas particularidades se centra la comparación y de los representantes.

**ESTO NO VA, VA A IR EN LAS GRAFICAS Y TABLA, LO PONGO PARA SEGUIR HABLANDO DE LOS SUBGRUPOS Y QUE NO SE PIERDA EN WSP**
**Hay que darle una 2da pasada a las tablas por si faltó agarrar algo que yo me perdí**
##### Iris:
###### Buenos resultados: (Metricas en general arriba de 0.95)
    1.  18  Bosque  C45 GainRatio   F: 0.955
    2.  7,8,9   Arbol   C45 Any     F: 0.945~
###### Peores resultados: (Metricas en general por abajo del 0.88)
    1.  15  Bosque  Var Impurity    F: 0.865    Calificó mal aunque tiene algunas metricas bastante bien
    2.  1,2,3   Arbol   Fijos   Any     F: 0.87 Dieron los falloffs más altos

##### Covertype:
###### Buenos resultados: (F-Measure mayor a 0.5)
    1.  D, F    Arbol   C45 Gain, Imp   F: 0.55 Resultados re identicos. Fueron los que mejor calificaron de 4-7.
    2.  E   Arbol   C45 GainRatio   F: 0.51 Tiene mejor promedio ponderado porque califica mejor 1-3, no califica 4-7 horriblemente pero en gral tiene peor performance que D y F
    3.  L   Bosque  C45 GainRatio   F: 0.45 MEjor precisión en 1-2, deteriora bastante la precisión en 3-7. El problema principal aca fue el falloff.
    4.  J,K Bosque  C45 Gain, Imp   F: 0.42 Resultados en general peores a L pero sin ninguna varianza significativa respecto al mismo
###### Peores resultados: (F-Measure mean  0.35, weights distribuidos no atipicamente (0.61-0.68))
    1.  G,H,I   Bosque  Fijos   Any (H GainRatio fue el peor). Califican demasiado mal clases 4-7. Se observa que los promedios ponderados no son atipicos porque estos casos son la minoria.

Se adjunta script de graficas

<div style="margin-top: 16px; margin-bottom: 16px;">
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/instancia1.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.17 - Partidas ganadas para cada modelo entrenado de la instancia 1</label>
    </div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/instancia2.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.18 - Partidas ganadas para cada modelo entrenado de la instancia 2</label>
    </div>
</div>

#### 3.4.2. Comparación
***
Aca hablamos de los mejores y los comparamos

**Observaciones generales:**
- C4.5 siempre dio mejores resultados que particiones en intervalos fijos. Esto no es de extrañarse dado que C45 es en si una mejora sobre ID3.
- Para *Covertype* se obtuvieron consistentemente mejores resultados empleando árboles que bosques.
- *Covertype* es un conjunto dificil de calificar, pese a contar con una muestra extremadamente grande de datos en el entrenamiento, los resultados obtenidos mediante la construcción de un árbol de decisión son fuertemente perjudicados a causa de la complejidad de los atributos, así como por la enorme proporción de elementos de clase 1, 2 y 3 frente a los pocos elementos de clase 4, 5, 6 y 7.
- Algunos resultados tuvieron una gran diferencia entre sus promedios generales y promedios ponderados (siendo los últimos bastante más altos). Esto es debido a que estos conjuntos son de los que menos éxito tienen al calificar elementos de la clase 4, 5, 6 y 7; pero a causa de su baja proporción respecto a los elementos de la clase 1, 2 y 3 los promedios ponderados no se ven tan disminuidos.
- En *Covertype* los bosques califican los elementos de clase 1, 2 y 3 mejor que los árboles. Sin embargo, tienen una muy baja Precisión y muy alto FallOff para los elementos de clase 4, 5, 6 y 7. Esto se debe a que para una cantidad no despreciable de clasificaciones (7) y la reducida cantidad de datos de elementos de clase 4, 5, 6 y 7; es muy dificil clasificar elementos booleanamente para cada clase, la mayoría de clasificaciones acaban en votaciones (a causa de que no ocurre la situación ideal de que 1 árbol de true y los otros false) y en cualquier árbol un false dice muy poca información (no tiene caracteristicas de elementos de esta clase, pero puede tener caracteristicas de cualquier otro). Esto para un conjunto de atributos y clasificaciones con alta complejidad acaba disminuyendo enormemente la calidad del árbol cuando no es facil detectar atributos con muy baja impureza que determinen fuertemente la pertenencia a una clase.

**Interpretaciones:**
- Si bien llegaron a formar parte de subconjuntos distintos, la medida no tuvo gran impacto en la performance de los modelos. Inclusive es imposible afirmar con el número de resultados obtenidos si estos subconjuntos son correctos para la métrica y no se dieron aleatoriamente por la separación en el conjuntos de validación y entrenamiento.
- Muchos de los resultados en *covertype* no necesariamente reflejan la realidad, esto es debido a la aleatoriedad de la validación (80/20). Una validación cruzada en 10 intervalos (Como la realizada en Iris) podría dar un intervalo de certeza mucho mayor sobre las conjeturas basadas en las interpretaciones de los candidatos.
- El haber desecho el one-hot encoding es un buen candidato a culpar por los malos resultados obtenidos a la hora de clasificar elementos del conjunto 4 al 7 en Covertype. La cantidad de elementos pertenecientes a estos conjuntos es muy baja y poder diferenciar con mejor granularidad y menor sesgo inducido sobre el tipo de suelo (soil_type) pudo haber ayudado a encontrar relaciones entre el mismo y la calificación del terreno.

## 4. Conclusiones
***

#### 4.1. Respecto a los Modelos
***
Acá hablar sobre la demora de los bosques y la dificultad de establecer una votación que estuviera buena. La randomness al empatar es bastante mierda y mete ruido.

#### 4.2. Respecto a los Atributos
***
Acá hablar de que el one hot encoding no rinde pa arboles

#### 4.3. Respecto a los Parámetros
***
Acá hablar de que los arboles no encaran tanto en continuos, fixed es caca, c45 y variables son mejores pero para un dataset tan grande nos cagó.

#### 4.4. Respecto a los Resultados
***
Acá hablar de que covertype da peores resultados porque es un dataset muy grande. Más conclusiones viendo los resultados. Hablar de que los promedios ponderados nos dieron mejor info que los promedios generales porque hay mucha cantidad de b1 en covertype y poca de otros como b4.

#### 4.5. Posibles mejoras
***
A
- Usar todos los valores en variables y c4.5
- Usar distintas measures en c4.5
- Mantener el onehotencoding o al menos no tratar como continuos esos atributos
